# MXResNet.

> MXResNet model.

MXResNet model, disscussed at https://forums.fast.ai/t/how-we-beat-the-5-epoch-imagewoof-leaderboard-score-some-new-techniques-to-consider

In [ ]:
#hide
from fastcore.test import *
import torch


In [ ]:
from functools import partial

from model_constructor.activations import Mish
from model_constructor.net import Net

# MXResNet constructor.

In [ ]:
mxresnet  = Net(stem_sizes = [3, 32, 64, 64], name='MXResNet', act_fn=Mish())

In [ ]:
mxresnet

MXResNet constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False
  sa: False, se: False
  stem sizes: [3, 32, 64, 64], stide on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [ ]:
mxresnet.block_sizes, mxresnet.layers

([64, 64, 128, 256, 512], [2, 2, 2, 2])

In [ ]:
#collapse_output
mxresnet.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish()
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish()
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish()
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mxresnet.stem(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
#collapse_output
mxresnet.body

Sequential(
  (l_0): Sequential(
    (bl_0): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish()
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): Mish()
    )
    (bl_1): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish()
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), str

In [ ]:
#collapse_output
mxresnet.body

Sequential(
  (l_0): Sequential(
    (bl_0): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish()
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): Mish()
    )
    (bl_1): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish()
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), str

In [ ]:
#collapse_output
mxresnet.head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten()
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mxresnet()(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size"

torch.Size([16, 1000])


In [ ]:
mxresnet_parameters = {'stem_sizes': [3, 32, 64, 64], 'act_fn': Mish()}
mxresnet34 = partial(Net, name='MXResnet32', expansion=1, layers=[3, 4, 6, 3], **mxresnet_parameters)
mxresnet50 = partial(Net, name='MXResnet50', expansion=4, layers=[3, 4, 6, 3], **mxresnet_parameters)

In [ ]:
model = mxresnet50(c_out=10)

In [ ]:
model

MXResnet50 constructor
  c_in: 3, c_out: 10
  expansion: 4, groups: 1, dw: False
  sa: False, se: False
  stem sizes: [3, 32, 64, 64], stide on 0
  body sizes [64, 128, 256, 512]
  layers: [3, 4, 6, 3]

In [ ]:
model.c_out, model.layers

(10, [3, 4, 6, 3])

# end
model_constructor
by ayasyrev